In [143]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [147]:
#degrees of freedom
DOFs = int(input("Enter degrees of freedom"))

theta = np.zeros((DOFs))
alpha = np.zeros((DOFs))
a = np.zeros((DOFs))
d = np.zeros((DOFs))

for i in range(0, DOFs):
  print("data for link", i)
  theta[i] = float(input("Theta"))
  alpha[i] = float(input("Alpha"))
  a[i] = float(input("a"))
  d[i] = float(input("d"))

Enter degrees of freedom3
data for link 0
Theta0
Alpha-1.05
a0
d5
data for link 1
Theta-1.05
Alpha-1.05
a0
d6
data for link 2
Theta1.05
Alpha1.05
a0
d4


In [150]:
DH_mat = np.hstack((np.hstack((a.T, alpha.T)), np.hstack((d.T,theta.T))))
print(DH_mat)

[ 0.    0.    0.   -1.05 -1.05  1.05  5.    6.    4.    0.   -1.05  1.05]


In [151]:
joint = [0]*DOFs
for i in range(0, DOFs):
  print("Type of joint", i+1)
  val = input()
  if val == "p":
    joint[i] = "p"
  else:
    joint[i] = "r"

Type of joint 1
p
Type of joint 2
p
Type of joint 3
p


In [153]:
#Transformation Matrix
temp = np.identity((4))
T = np.zeros((DOFs, 4, 4))

for i in range(0, DOFs):
  costheta = math.cos(theta[i])
  sintheta = math.sin(theta[i])
  cosalpha = math.cos(alpha[i])
  sinalpha = math.sin(alpha[i])

  T[i, :, :] = [[costheta, -sintheta*cosalpha, sintheta*sinalpha, a[i]*costheta], [sintheta, costheta*cosalpha, -costheta*sinalpha, a[i]*sintheta], [0, sinalpha, cosalpha, d[i]], [0, 0, 0, 1]]

print(T)

[[[ 1.         -0.         -0.          0.        ]
  [ 0.          0.49757105  0.86742323  0.        ]
  [ 0.         -0.86742323  0.49757105  5.        ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.49757105  0.43160468  0.75242305  0.        ]
  [-0.86742323  0.24757695  0.43160468 -0.        ]
  [ 0.         -0.86742323  0.49757105  6.        ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.49757105 -0.43160468  0.75242305  0.        ]
  [ 0.86742323  0.24757695 -0.43160468  0.        ]
  [ 0.          0.86742323  0.49757105  4.        ]
  [ 0.          0.          0.          1.        ]]]


In [154]:
temp = np.identity((4))
Z = [0]*(DOFs + 1)
Z[0] = np.array([[0], [0], [1]])
O = [0]*(DOFs + 1)
O[0] = np.array([0, 0, 0])

for i in range(DOFs):
  temp = np.matmul(temp,T[i])
  Z[i+1] = np.matmul(temp[0:3, 0:3],Z[0])
  O[i+1] = temp[0:3, 3].T

print(temp)

[[ 0.62196087  0.54477061  0.56248525  3.00969221]
 [-0.76056786  0.59116482  0.26844123  7.78997407]
 [-0.1862826  -0.59476814  0.7820164   7.47819837]
 [ 0.          0.          0.          1.        ]]


In [155]:
J = [0]*DOFs
for i in range(DOFs):
  if joint[i] == 'p':
    J[i] = np.concatenate((Z[i], np.array([[0,0,0]]).T), axis=0)

  else:
    mat = np.cross(Z[i].T, (O[DOFs]-O[i]).T)
    J[i]=np.concatenate((mat.T,Z[i]),axis=0)

jacobian = J[0]
for i in range(1, DOFs):
  jacobian = np.concatenate((jacobian,J[i]),axis=1)

print("Jacobian matrix=")
print(jacobian)

Jacobian matrix=
[[ 0.          0.          0.75242305]
 [ 0.          0.86742323  0.64635868]
 [ 1.          0.49757105 -0.12680698]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


In [136]:
#Finding end effector coordinates
final_T = T[0]
for i in range(1, DOFs):
  final_T = np.matmul(final_T, T[i])

end_effector = final_T[0:3, -1]
print(end_effector)

[-0.67464112 -0.25174844 -7.99998985]


In [141]:
#Finding end effector velocity
q_dot = np.zeros((3, 1))
print("Insert vector of joint angular velocity, each component after pressing enter (x,y,z)")
for i in range(0,3):
  q_dot[i] = float(input())

Insert vector of joint angular velocity, each component after pressing enter (x,y,z)
3
2
3


In [142]:
velocity = np.matmul(jacobian[0:3, :].T, q_dot)
omega = np.matmul(jacobian[3:6, :].T, q_dot)
print(velocity)
print(omega)

[[-5.94036920e-01]
 [ 1.72068928e+01]
 [-3.00548145e+00]
 [-2.86635949e-17]]
[[ 3.        ]
 [ 3.        ]
 [ 0.        ]
 [-3.00548145]]
